In [1]:
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import pandas as pd
import json
import csv
import sys
import time

ckey = 'bXRPnBdpsgRdrkfbrHBHFInjl'
csecret = 'xm93rCpGvtV6E2lwx6392l5uUfYpz9dgCe7zREGtCWeWd5b6WB'
atoken = '1191115298788777987-qUAhRZ5dAw2wN1QewZxjROvtnVLHBs'
asecret = '595HhAG8XXlbN7BPYfLVUio1ozLqmLMnFDDIaHfzPmeSv'

def toDataFrame(tweets):

    DataSet = pd.DataFrame()

    DataSet['tweetID'] = [tweet.id for tweet in tweets]
    DataSet['tweetText'] = [tweet.text for tweet in tweets]
    DataSet['tweetRetweetCt'] = [tweet.retweet_count for tweet in tweets]
    DataSet['tweetFavoriteCt'] = [tweet.favorite_count for tweet in tweets]
    DataSet['tweetSource'] = [tweet.source for tweet in tweets]
    DataSet['tweetCreated'] = [tweet.created_at for tweet in tweets]
    DataSet['userID'] = [tweet.user.id for tweet in tweets]
    DataSet['userScreen'] = [tweet.user.screen_name for tweet in tweets]
    DataSet['userName'] = [tweet.user.name for tweet in tweets]
    DataSet['userCreateDt'] = [tweet.user.created_at for tweet in tweets]
    DataSet['userDesc'] = [tweet.user.description for tweet in tweets]
    DataSet['userFollowerCt'] = [tweet.user.followers_count for tweet in tweets]
    DataSet['userFriendsCt'] = [tweet.user.friends_count for tweet in tweets]
    DataSet['userLocation'] = [tweet.user.location for tweet in tweets]
    DataSet['userTimezone'] = [tweet.user.time_zone for tweet in tweets]
    DataSet['Coordinates'] = [tweet.coordinates for tweet in tweets]
    DataSet['GeoEnabled'] = [tweet.user.geo_enabled for tweet in tweets]
    DataSet['Language'] = [tweet.user.lang for tweet in tweets]
    tweets_place= []
    #users_retweeted = []
    for tweet in tweets:
        if tweet.place:
            tweets_place.append(tweet.place.full_name)
        else:
            tweets_place.append('null')
    DataSet['TweetPlace'] = [i for i in tweets_place]
    #DataSet['UserWhoRetweeted'] = [i for i in users_retweeted]

    return DataSet

OAUTH_KEYS = {'consumer_key':ckey, 'consumer_secret':csecret,'access_token_key':atoken, 'access_token_secret':asecret}
auth = tweepy.OAuthHandler(OAUTH_KEYS['consumer_key'], OAUTH_KEYS['consumer_secret'])
#auth = tweepy.AppAuthHandler('key', 'key')

api = tweepy.API(auth, wait_on_rate_limit=True,wait_on_rate_limit_notify=True)
if (not api):
    print ("Can't Authenticate")
    sys.exit(-1)
else:
# I am trying to download from Dec 1st to Dec 7th but I am not able to

    cursor = tweepy.Cursor(api.search, q='#OurBestDaysStillLieAhead',lang='en')
    results=[]
    for item in cursor.items(10000):
        results.append(item)

    DataSet = toDataFrame(results)

In [2]:
DataSet["text"] = DataSet["tweetText"]
DataSet.shape

(0, 20)

In [3]:
import re
import preprocessor as p

DataSet["text"] = DataSet["tweetText"]

pd.options.display.max_colwidth = 500

def preprocess_tweet(row):
    text = row['text']
    text = p.clean(text)
    return text

DataSet["text"]  = DataSet.apply(preprocess_tweet, axis=1)

DataSet["text"] = DataSet["text"].str.lower().str.replace('[^\w\s]',' ').str.replace('\s\s+', ' ')


DataSet[["tweetText", "text"]].head()

AttributeError: Can only use .str accessor with string values!

In [ ]:
from nltk.tokenize import word_tokenize



def custom_tokenize(text):
    if not text:
        print('The text to be tokenized is a None type. Defaulting to blank string.')
        text = ''
    return word_tokenize(text)
DataSet['text2'] = DataSet['text'].apply(custom_tokenize)


DataSet[["text", "text2"]].head()


In [ ]:
from nltk.stem.snowball import SnowballStemmer

# Use English stemmer.
stemmer = SnowballStemmer("english")

DataSet['text3'] = DataSet['text2'].apply(lambda x: [stemmer.stem(y) for y in x]) 


DataSet[["tweetText", "text2", "text3"]].head()

In [ ]:
import datetime
year = datetime.date.today().year
month = datetime.date.today().month
month = '{:02d}'.format(month)
day = datetime.date.today().day
day = '{:02d}'.format(day)

today = datetime.date.today().strftime("%Y.%m.%d")
DataSet["keresoszo"] = "#OurBestDaysStillLieAhead"
DataSet["letoltes_datum"] = today

In [ ]:
DataSet.to_csv('data/OurBestDaysStillLieAhead' + str(year) + str(month) + str(day) + '.csv',index=False)

In [ ]:
DataSet.head()